# Tutorial 4

## Mean Squared Displacement (MSD)


Molecules in liquds, gases and solids do not stay in the same place and move constantly. Think about a drop of dye in a glass of water, as time passes the dye distributes throughout the water. This process is called diffusion and is common throughout nature and an incredibly relevant property for materials scientists who work on things like batteries.  

Using the dye as an example, the motion of a dye molecule is not simple. As it moves it is jostled by collisions with other molecules, preventing it from moving in a straight path. If the path is examined in close detail, it will be seen to be a good approximation to a random walk. In mathmatics a random walk is a series of steps, each taken in a random direction. This was analysed by Albert Einstein in a study of Brownian motion and he showed that the mean square of the distance travelled by a particle following a random walk is proportional to the time elapsed. 

\begin{align}
\Big \langle r_{i}^{2} \big \rangle & = 6 D_t + C 
\end{align}


where 

\begin{align}
\Big \langle r_{i}^{2} \big \rangle = \frac{1}{3} \Big< | r_{i}(t) - r_{i}(0) |^2 \Big>.
\end{align}


where $\Big \langle r^2 \big \rangle$ is the mean squared distance, t is time, $D_t$ is the diffusion rate and C is a constant. If $\Big \langle r_{i}^{2} \big \rangle$ is plotted as a function of time, the gradient of the curve obtained is equal to 6 times the self-diffusion coefficient of particle i. 
The state of the matter effects the shape of the MSD plot, solids, where little to no diffusion is occuring, has a flat MSD profile. In a liquid however, the particles diffusion randomly and the gradient of the curve is proportional to the diffusion coefficient. 

## What is the mean squared displacement

Going back to the example of the dye in water, lets assume for the sake of simplicity that we are in one dimension. Each step can either be forwards or backwards and we cannot predict which. From a given starting position, what distance is our dye molecule likely to travel after 1000 steps? This can be determined simply by adding together the steps, taking into account the fact that steps backwards subtract from the total, while steps forward add to the total. Since both forward and backward steps are equally probable, we come to the surprising conclusion that the probable distance travelled sums up to zero.

By adding the square of the distance we will always be adding positive numbers to our total which now increases linearly with time. Based upon equation 1 it should now be clear that a plot of $\Big \langle r_{i}^{2} \big \rangle$ vs time with produce a line, the gradient of which is equal to 6D. Giving us direct access to the diffusion coefficient of the system. 

In [1]:
from polypy import read as rd
from polypy import msd as msd
from polypy import utils as ut
from polypy import write as wr
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import corner
from sklearn.utils import resample
from uravu.relationship import Relationship
from uravu import plotting, utils

In [2]:
import emcee
import numpy as np

def straightline(x, m, c):
    """ 
    Calculate y for a straight line
    
    Args:
        x (list): the abscissa values at which the data/model is defined
        m (float): the gradient of the straight line
        c (float): the intercept of the straight line
    """
    return m * x + c

def loglikelihood(theta, y, dy, x):
    """
    The natural logarithm of the joint likelihood, equation from 
    DOI: 10.1107/S1600576718017296
    
    Args:
        theta (tuple): a sample containing individual parameter values
        y (list): the set of data/observations
        dy (list): the standard deviation of the data points
        x (list): the abscissa values at which the data/model is defined
    """
    
    m, c = theta
    
    md = straightline(x, m, c)
    
    return -0.5*np.sum(((md - y)/dy)**2 + np.log(2 * np.pi * dy ** 2))

def mcmc_sample(y, x, dy, m, c, n_samples=500, n_burn=500):
    """
    The MCMC sampler using emcee

    Args:
        y (list): the set of data/observations
        dy (float): the standard deviation of the data points
        x (list): the abscissa values at which the data/model is defined
        m (float): an initial guess of the gradient
        c (float): an initial guess of the intercept
        n_samples (int): number of samples
        n_burn (int): number of burn in samples (will be ignored)
    """
    walkers = 100  

    m_min = m - 10. * m
    m_max = m + 10. * m

    mini = np.random.uniform(m_min, m_max, walkers) 

    c_min = c - 10. * c  
    c_max = c + 10. * c  

    cini = np.random.uniform(c_min, c_max, walkers)

    inisamples = np.array([mini, cini]).T 

    ndims = inisamples.shape[1] 

    argslist = (y, dy, x)

    sampler = emcee.EnsembleSampler(walkers, ndims, loglikelihood, args=argslist)

    sampler.run_mcmc(inisamples, n_samples + n_burn);

    postsamples = sampler.chain[:, n_burn:, :].reshape((-1, ndims))

    return postsamples, sampler, ndims

This example will use a short (50,000 steps), pre-prepared trajectory of bulk $CaF_2$. In reality we probably want a considerably longer simulation (~10,000,000 steps). Such simulations generate huge files (5GB) and the analysis would take too long for this tutorial. 

The first step is to read the history file to generate the data. The function read_history expects two things, the filename of the history file and a list of atoms to read. It will return a dictionary containing the atom labels, trajectories, lattice vectors, timesteps and number of atoms.

```
data = {'label': Atom Names,
        'trajectories': Atomic trajectories,
        'lv': Lattice vectors,
        'timesteps': Number of timesteps,
        'natoms': Number of Atoms}
```

In [3]:
#data = rd.read_history("../../../../HISTORY", ["PA"])
#print(data['frac_trajectories'])
#print(data['trajectories'])


data = rd.read_history("../example_data/HISTORY", ["F"])

print(data['timesteps'])

500


Once the data has been read into the code the MSD calculation can be performed. The msd function requires the data and the time between records which in this case is 0.25ps. The msd function will then return a dictionary containing the time, 3D MSD and the directional components. 

```
data = {'msd': 3D MSD,
        'xmsd': MSD in the X direction,
        'ymsd': MSD in the Y direction,
        'zmsd': MSD in the Z direction,
        'time': Time}
```

In [4]:
msd_data = msd.msd(data, timestep=0.1)

(1000, 3)


In [5]:
=

SyntaxError: invalid syntax (<ipython-input-5-9ae22869553b>, line 1)

This data can then be plotted using the plot function. 

In [ ]:
wr.msd_plot(msd_data)

In [ ]:
plt.scatter(msd_data['sd_time'], msd_data['squared_displacements'], s=10, alpha=0.2)

In [ ]:
plt.hist(msd_data['squared_displacements'][-data['natoms']:-1], bins=100)
plt.axvline(np.mean(msd_data['squared_displacements'][-data['natoms']:-1]), ls="--")

In [ ]:
dataset = msd_data['squared_displacements']
dataset = np.split(dataset, data['timesteps']-1)

In [ ]:
confidence_intervals = []
means = []
all_data = np.zeros((data['timesteps']-1, 1000))
all_means = []
all_times = []
errors = []
for i in range(data['timesteps']-1):
    sampled_means = [np.mean(resample(dataset[i], replace=True, n_samples=100)) for j in range(1000)]
    means.append(np.mean(sampled_means))
    all_data[i,:] = np.array(sampled_means)
    all_means.append(sampled_means)
    errors.append(np.std(sampled_means))
    confidence_intervals.append([np.percentile(sampled_means, 2.5), np.percentile(sampled_means, 97.5)])



In [ ]:
all_means = np.asarray(all_means)
plt.plot(msd_data['sd_time'], all_means.flatten(), '.', alpha=0.005)
plt.show()

In [ ]:
from scipy.stats import linregress

ci_arr = np.array(confidence_intervals).T
y_error = np.array([msd_data['msd'] - ci_arr[0], ci_arr[1] - msd_data['msd']])
plt.errorbar(msd_data['time'], msd_data['msd'], yerr=y_error, alpha=0.2)
res = linregress(msd_data['time'], msd_data['msd'])
ps, sampler, ndims = mcmc_sample(msd_data['msd'], msd_data['time'], y_error, res.slope, res.intercept, 2000, 2000)
a = np.random.choice(ps.shape[0], size=100)
for k in ps[a]:
    plt.plot(msd_data['time'], straightline(msd_data['time'], k[0], k[1]), 'tab:green', alpha=0.05)

plt.title(i)
plt.show()
fig = corner.corner(ps, labels=[r"$m$", r"$c$"])
plt.show()


In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain(discard=100)
labels = ["m", "c"]
for i in range(ndims):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
#mcmc = np.percentile(samples[:, i], [16, 50, 84])
print(np.mean(np.mean(samples, axis=0), axis=0), np.std(np.std(samples, axis=0), axis=0))

Using the data the diffusion coefficient can then be calculated from the slopes. 

In [ ]:
Diff = ut.linear_regression(msd_data['time'], msd_data['msd'])[0]
Diffusion_Coefficient = ut.three_d_diffusion_coefficient(Diff)

XYDiff = ut.linear_regression(msd_data['time'], msd_data['xymsd'])[0]
XYDiffusion_Coefficient = ut.two_d_diffusion_coefficient(XYDiff)#

XZDiff = ut.linear_regression(msd_data['time'], msd_data['xzmsd'])[0]
XZDiffusion_Coefficient = ut.two_d_diffusion_coefficient(XZDiff)

YZDiff = ut.linear_regression(msd_data['time'], msd_data['yzmsd'])[0]
YZDiffusion_Coefficient = ut.two_d_diffusion_coefficient(YZDiff)

XDiff = ut.linear_regression(msd_data['time'], msd_data['xmsd'])[0]
XDiffusion_Coefficient = ut.one_d_diffusion_coefficient(XDiff)#

YDiff = ut.linear_regression(msd_data['time'], msd_data['ymsd'])[0]
YDiffusion_Coefficient = ut.one_d_diffusion_coefficient(YDiff)

ZDiff = ut.linear_regression(msd_data['time'], msd_data['zmsd'])[0]
ZDiffusion_Coefficient = ut.one_d_diffusion_coefficient(ZDiff)

In [ ]:
print("Three Dimensional Diffusion Coefficient", Diffusion_Coefficient)
print("Two Dimensional Diffusion Coefficient in XY", XYDiffusion_Coefficient)
print("Two Dimensional Diffusion Coefficient in XZ", XZDiffusion_Coefficient)
print("Two Dimensional Diffusion Coefficient in YZ", YZDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in X", XDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Y", YDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Z", ZDiffusion_Coefficient)

In [ ]:
volume = np.mean(ut.system_volume(data))

In [ ]:
XYZCond = ut.conductivity(data['natoms'], volume, Diffusion_Coefficient, 1500)
print("Three Dimensional Conductivity", XYZCond)
XYCond = ut.conductivity(data['natoms'], volume, XYDiffusion_Coefficient, 1500)
print("Two Dimensional Conductivity in XY", XYCond)
XZCond = ut.conductivity(data['natoms'], volume, XZDiffusion_Coefficient, 1500)
print("Two Dimensional Conductivity in XZ", XZCond)
YZCond = ut.conductivity(data['natoms'], volume, YZDiffusion_Coefficient, 1500)
print("Two Dimensional Conductivity in YZ", YZCond)
XCond = ut.conductivity(data['natoms'], volume, XDiffusion_Coefficient, 1500)
print("One Dimensional Conductivity in X", XCond)
YCond = ut.conductivity(data['natoms'], volume, YDiffusion_Coefficient, 1500)
print("One Dimensional Conductivity in Y", YCond)
ZCond = ut.conductivity(data['natoms'], volume, ZDiffusion_Coefficient, 1500)
print("One Dimensional Conductivity in Z", ZCond)

#### Smoothing the MSD

This example has used one sweep of the trajectory, meaning that the MSD has been calculated from one value of $r_{i}^{0}$. In order to increase your statistics and sharpen your  MSD plot it is often better to use multiple values of $r_{i}^{0}$. Basically, use multiple starting points. The smooth_msd function allows you to increase the number of runs with the runs parameter.

In [ ]:
#smsd_data = msd.smooth_msd(data, 0.1, runs=5)

np.savetxt("squared_displacements.csv", np.array([smsd_data['sd_time'], smsd_data['squared_displacements']]), delimiter=",")

In [ ]:
tp = np.genfromtxt("squared_displacements.csv", delimiter=",")
plt.scatter(tp[0], tp[1], alpha=0.1)

The data can then be plotted, and clearly the line is less noisey and thus the gradient is much more accurate. 

In [ ]:
wr.msd_plot(smsd_data)

In [ ]:
plt.scatter(smsd_data['sd_time'], smsd_data['squared_displacements'], s=10, alpha=0.2)

In [ ]:
plt.hist(smsd_data['squared_displacements'][-data['natoms']:-1], bins=100)
plt.axvline(np.mean(smsd_data['squared_displacements'][-data['natoms']:-1]), ls="--")



In [ ]:
xy = {'time': smsd_data['sd_time'], 'sd': smsd_data['squared_displacements']}
dataset = pd.DataFrame(xy).sort_values(by='time')

In [ ]:
confidence_intervals = []
means = []
all_data = np.zeros((data['timesteps']-1, 1000))
all_times = []
all_means = []
errors = []
count = 0
for time, sd in dataset.groupby('time'):
    times = np.zeros(1000) + time
    all_times.append(times.tolist())
    sampled_means = [np.mean(resample(sd['sd'], replace=True, n_samples=100)) for j in range(1000)]
    means.append(np.mean(sampled_means))
    all_data[count,:] = np.array(sampled_means)
    
    all_means.append(sampled_means)
    errors.append(np.std(sampled_means))
    confidence_intervals.append([np.percentile(sampled_means, 2.5), np.percentile(sampled_means, 97.5)])
    count = count + 1
    

all_means = np.asarray(all_means)
all_times = np.asarray(all_times)

plt.plot(all_times.flatten(), all_means.flatten(), '.', alpha=0.005)
plt.show()

In [ ]:
from scipy.stats import linregress

ci_arr = np.array(confidence_intervals).T
y_error = np.array([smsd_data['msd'] - ci_arr[0], ci_arr[1] - smsd_data['msd']])
plt.errorbar(smsd_data['time'], smsd_data['msd'], yerr=y_error, alpha=0.2)
res = linregress(smsd_data['time'], smsd_data['msd'])
ps, sampler, ndims = mcmc_sample(smsd_data['msd'], smsd_data['time'], y_error, res.slope, res.intercept, 2000, 2000)
a = np.random.choice(ps.shape[0], size=100)
for k in ps[a]:
    plt.plot(smsd_data['time'], straightline(smsd_data['time'], k[0], k[1]), 'tab:green', alpha=0.05)

plt.title(i)
plt.show()
fig = corner.corner(ps, labels=[r"$m$", r"$c$"])
plt.show()


In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain(discard=100)
labels = ["m", "c"]
for i in range(ndims):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
Diff = ut.linear_regression(smsd_data['time'], smsd_data['msd'])[0]
Diffusion_Coefficient = ut.three_d_diffusion_coefficient(Diff)

XDiff = ut.linear_regression(smsd_data['time'], smsd_data['xmsd'])[0]
XDiffusion_Coefficient = ut.three_d_diffusion_coefficient(XDiff)

YDiff = ut.linear_regression(smsd_data['time'], smsd_data['ymsd'])[0]
YDiffusion_Coefficient = ut.three_d_diffusion_coefficient(YDiff)

ZDiff = ut.linear_regression(smsd_data['time'], smsd_data['zmsd'])[0]
ZDiffusion_Coefficient = ut.three_d_diffusion_coefficient(ZDiff)

In [ ]:
print("Three Dimensional Diffusion Coefficient", Diffusion_Coefficient, "")
print("One Dimensional Diffusion Coefficient in X", XDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Y", YDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Z", ZDiffusion_Coefficient)

## Arrhenius


It is then possible to take diffusion coefficients, calculated over a large temperature range and, using the Arrhenius equation calculate the activation energy for diffusion. Common sense and chemical intuition suggest that the higher the temperature, the faster a given chemical reaction will proceed. Quantitatively this relationship between the rate a reaction proceeds and its temperature is determined by the Arrhenius Equation. At higher temperatures, the probability that two molecules will collide is higher. This higher collision rate results in a higher kinetic energy, which has an effect on the activation energy of the reaction. The activation energy is the amount of energy required to ensure that a reaction happens.  
  
\begin{align}
k = A * e^{(-Ea / RT)}
\end{align}
  
where k is the rate coefficient, A is a constant, Ea is the activation energy, R is the universal gas constant, and T is the temperature (in kelvin).



## Ionic Conductivity

Usefully, as we have the diffusion coefficient, the number of particles (charge carriers) and the ability to calculate the volume, we can convert this data into the ionic conductivity and then the resistance. 

\begin{align}
\sigma & = \frac{D C_F e^2}{k_B T} 
\end{align}

where $\sigma$ is the ionic conductivity, D is the diffusion coefficient, $C_F$ is the concentration of charge carriers, which in this case if F ions, $e^2$ is the charge of the diffusing species, $k_B$ is the Boltzmann constant and T is the temperature. 

The resitance can then be calculated according to 

\begin{align}
\Omega & = \frac{1}{\sigma} 
\end{align}

So the first step is to calculate the volume, the system voume module will do this from the given data. 

In [ ]:
volume = np.dot(data['lv'][-1][0,:], np.cross(data['lv'][-1][1,:], data['lv'][-1][2,:] ))

The number of charge carriers is just the total number of atoms.

In [ ]:
Number_of_Charge_Carriers = traj_obj.get_natoms("F")

Then the conductivity can be calculated by the conductivity module, note that the temperature of this simulation was 1500 K. 

In [ ]:
Ionic_Conductivity = ut.conductivity(Number_of_Charge_Carriers, volume, Diffusion_Coefficient, 1500)

In [ ]:
print("Ionic Conductivity :", Ionic_Conductivity)

In [ ]:
print("Resistivity :", (1 / Ionic_Conductivity)) 

## Simulation Length

It is important to consider the lenght of your simulation (Number of steps). The above examples use a short trajectory but it is at a sufficient temperature that there are enough diffusion events to get a good MSD plot. The following example is of a very short simulation, you will hopefully note that the MSD plot is clearly not converged. 

In [ ]:
data_short = rdpl.read_trajectory("../example_data/HISTORY_short")
short_traj = trajectory.PolyTrajectory(data_short)

In [ ]:
msd_data = msd.msd(data_short, 0.1)

In [ ]:
wr.msd_plot(msd_data)

Clearly the diffusion coefficient is wrong. But without viewing the plot it is hard to tell. 

In [ ]:
Diff = ut.linear_regression(msd_data['time'], msd_data['msd'])
Diffusion_Coefficient = ut.three_d_diffusion_coefficient(Diff)

XDiff = ut.linear_regression(msd_data['time'], msd_data['xmsd'])
XDiffusion_Coefficient = ut.three_d_diffusion_coefficient(XDiff)

YDiff = ut.linear_regression(msd_data['time'], msd_data['ymsd'])
YDiffusion_Coefficient = ut.three_d_diffusion_coefficient(YDiff)

ZDiff = ut.linear_regression(msd_data['time'], msd_data['zmsd'])
ZDiffusion_Coefficient = ut.three_d_diffusion_coefficient(ZDiff)

In [ ]:
print("Three Dimensional Diffusion Coefficient", Diffusion_Coefficient, "")
print("One Dimensional Diffusion Coefficient in X", XDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Y", YDiffusion_Coefficient)
print("One Dimensional Diffusion Coefficient in Z", ZDiffusion_Coefficient)

## State of Matter

It is possible to identify the phase of matter from the MSD plot.

<center>
    <br>
    <img src="./figures/fig001.png" width="400px">
    <i>Figure 1. The anticipated MSD form for each state of matter.</i>
    <br>
</center>

The Fluorine diffusion discussed already clearly shows that the fluorine sub lattice has melted and the diffusion is liquid like. Whereas, carrying out the same analysis on the Calcium sub lattice shows that while the fluorine sub lattice has melted, the Calcium sub lattice is still behaving like a solid. 




## Regional MSD Calculations

Often in solid state chemistry simulations involve defects, both structural e.g. grain boundaries, dislocations and surface, and chemical e.g. point defects. It is important to try and isolate the contributions of these defects to the overall properties. Regarding diffusion, it could be imagined that a certain region within a structure will have different properties compared with the stoichiometric bulk, e.g. a grain boundary vs the grains, or the surface vs the bulk. `polypy` has the capability to isolate trajectories that pass within certain regions of a structure and thus calculate a diffusion coefficient for those regions. 

In [ ]:
import matplotlib.pyplot as plt
msd_data, transport_data = msd.plane_msd(data, 0.1, runs=1, ul=30, ll=-30, direction="x")

print(transport_data)